## Условие

### Введение

Для выполнения задачи будет использован csv файл “**vgsale_1**”, содержащий данные о **видеоиграх, выпущенных с 1980 по 2020 гг.** Каждое из наблюдений в файле имеет 10 характеристик:

Name – название игры,
Platform – игровая платформа (PC, PSP, X360 и др.),
Year – год выпуска игры,
Genre – жанр игры,
Publisher – издатель игры,
NA_Sales – продажи в Северной Америке (в миллионах),
EU_Sales – продажи в Европе (в миллионах),
JP_Sales – продажи в Японии (в миллионах),
Other_Sales – продажи в остальных странах мира (в миллионах),
Global_Sales – объем продаж по всему миру.
Загрузите файл «vgsales_1.csv» в объект DataFrame, рассчитайте необходимые показатели и визуализируйте информацию, используя функции любой библиотеки для визуализации данных.

### Задание

1. Ответь на вопрос: Игры каких жанров были наиболее популярны до 2000 года, а какие после?Оцени популярность жанров по количеству выпущенных игр и по объему продаж по всему миру. Для визуализации полученных результатов используй столбчатые диаграммы.
2. Отобрази на графике общее число видеоигр, выпущенных в каждом году.
3. Определи трех издателей, выпустивших наибольшее количество видеоигр.Изобрази количество выпущенных издателями видеоигр для каждой платформы на столбчатой диаграмме (можно использовать диаграмму с накоплением).
4. Отобрази на круговых диаграммах доли суммарного объема продаж с 1980 г. до 2000 г. и с 2000 г. до 2020 г. в Северной Америке, Европе, Японии (также для построения корректных диаграмм используйте столбец “Other Sales”) от объема продаж по всему миру.

Примечание. Одна и та же игра может встречаться в выборке несколько раз, т.к. она может быть выпущена на нескольких платформах.

## Решение

Подготовка данных

In [ ]:
# Импорт необходимых библиотек
import numpy as np
import pandas as pd
import plotly.graph_objects as go

In [ ]:
# Скачиваем датасет
!wget --no-check-certificate "https://docs.google.com/uc?export=download&id=1XV9IZ8cGArgCO9VkGId_IW2fewhg_tLN" -O "vgsale_1.csv"

--2025-07-05 20:24:47--  https://docs.google.com/uc?export=download&id=1XV9IZ8cGArgCO9VkGId_IW2fewhg_tLN
Resolving docs.google.com (docs.google.com)... 173.194.210.138, 173.194.210.100, 173.194.210.101, ...
Connecting to docs.google.com (docs.google.com)|173.194.210.138|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1XV9IZ8cGArgCO9VkGId_IW2fewhg_tLN&export=download [following]
--2025-07-05 20:24:47--  https://drive.usercontent.google.com/download?id=1XV9IZ8cGArgCO9VkGId_IW2fewhg_tLN&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 172.217.204.132, 2607:f8b0:400c:c15::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|172.217.204.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1370032 (1.3M) [application/octet-stream]
Saving to: ‘vgsale_1.csv’

vgsale_1.csv        100%[===================>]   1.31M  --.-KB/s 

In [ ]:
# Открываем скачанный файл
df_origin = pd.read_csv("./vgsale_1.csv", sep=',')
df_origin['Year'] = df_origin['Year'].round().astype('Int64')
df_origin['Publisher'] = df_origin['Publisher'].astype(str)

In [ ]:
# Удалаяем строки, где не указан год выпуска и удаляем дубликаты игр на разных платформах
df_games = (df_origin.dropna(subset='Year')
           .groupby(['Name', 'Year', 'Genre', 'Publisher'], as_index=False)
           .agg({
           'NA_Sales': 'sum',
           'EU_Sales': 'sum',
           'JP_Sales': 'sum',
           'Other_Sales': 'sum',
           'Global_Sales': 'sum'
      })
)

In [ ]:
# Создаем датасет по жанрам для анализа и визуализации
summary = pd.DataFrame(index=df_games['Genre'].unique())

In [ ]:
# Группируем по жанрам и объему продаж
summary['Количество до 2000'] = df_games[df_games['Year'] < 2000].groupby('Genre')['Name'].count()
summary['Количество после 2000'] = df_games[df_games['Year'] >= 2000].groupby('Genre')['Name'].count()
summary['Продажи до 2000'] = df_games[df_games['Year'] < 2000].groupby('Genre')['Global_Sales'].sum()
summary['Продажи после 2000'] = df_games[df_games['Year'] >= 2000].groupby('Genre')['Global_Sales'].sum()
summary['Продажи на 1 игру до 2000'] = summary['Продажи до 2000'] / summary['Количество до 2000']
summary['Продажи на 1 игру после 2000'] = summary['Продажи после 2000'] / summary['Количество после 2000']

# Сортируем жанры по алфавиту
summary = summary.sort_index()

# Приводим год и количество игр к целому числу
summary[['Количество до 2000', 'Количество после 2000']] = summary[['Количество до 2000', 'Количество после 2000']].astype(int)

### Выполнение задания

In [ ]:
# Строим графики
# График 1: Количество игр по жанрам
fig1 = go.Figure()
fig1.add_bar(
    x=summary.index,
    y=summary['Количество до 2000'],
    name='До 2000 года'
)
fig1.add_bar(
    x=summary.index,
    y=summary['Количество после 2000'],
    name='После 2000 года'
)
fig1.update_layout(
    title='Количество выпущенных игр по жанрам',
    xaxis_title='Жанр',
    yaxis_title='Количество игр',
    barmode='group',
    title_x=0.5,
    width=900,
    height=300
)
fig1.show()

# График 2: Продажи по жанрам
fig2 = go.Figure()
fig2.add_bar(
    x=summary.index,
    y=summary['Продажи до 2000'],
    name='До 2000 года'
)
fig2.add_bar(
    x=summary.index,
    y=summary['Продажи после 2000'],
    name='После 2000 года'
)
fig2.update_layout(
    title='Суммарные мировые продажи по жанрам',
    xaxis_title='Жанр',
    yaxis_title='Продажи (млн копий)',
    barmode='group',
    title_x=0.5,
    width=900,
    height=300
)
fig2.show()

# График 3: Продажи на игру до 2000
fig3 = go.Figure()
fig3.add_bar(
    x=summary.index,
    y=summary['Продажи на 1 игру до 2000'],
    name='До 2000 года'
)

fig3.update_layout(
    xaxis={'categoryorder': 'total descending'},
    title='Продаж на 1 игру в среднем до 2000',
    xaxis_title='Жанр',
    title_x=0.5,
    width=900,
    height=300
)
fig3.show()

# График 4: Продажи на игру после 2000
fig4 = go.Figure()
fig4.add_bar(
    x=summary.index,
    y=summary['Продажи на 1 игру после 2000'],
    name='После 2000 года'
)

fig4.update_layout(
    xaxis={'categoryorder': 'total descending'},
    title='Продаж на 1 игру в среднем после 2000',
    xaxis_title='Жанр',
    title_x=0.5,
    width=900,
    height=300
)
fig4.show()

## Ответ

До 2000 года наиболее популярными жанрами были платформеры, головоломки и ролевые игры. После 2000 года лидерство захватили шутеры и спортивные игры, платформеры остались в топе. Скорее всего, это связано с ростом технчиеских возможностей (вычислительных и визуальных).

## Общее число видеоигр, выпущенных в каждом году

In [ ]:
games_per_year = df_games.groupby('Year')['Name'].count()
fig5 = go.Figure()
fig5.add_scatter(x=games_per_year.index, y=games_per_year.values)
fig5.update_layout(title='Число выпущенных игр по годам', xaxis_title='Год', yaxis_title='Количество игр')
fig5.show()

Похоже, что данные в таблице актуальны только для периода до 2015 года.

## Три издателя, выпустивших наибольшее количество видеоигр

In [ ]:
top_publishers = df_origin['Publisher'].value_counts().head(3).index
df_top = df_origin[df_origin['Publisher'].isin(top_publishers)]
platform_publisher = df_top.groupby(['Platform', 'Publisher'])['Name'].count().unstack(fill_value=0)


In [ ]:
fig = go.Figure()
for publisher in top_publishers:
    if publisher in platform_publisher.columns:
        fig.add_bar(x=platform_publisher.index, y=platform_publisher[publisher], name=publisher)
fig.update_layout(barmode='stack',
                  xaxis={'categoryorder': 'total descending'},
                  title='Выпущенные игры топ-3 издателей по платформам',
                  xaxis_title='Платформа',
                  yaxis_title='Количество игр')
fig.show()

## Доли суммарного объема продаж с 1980 г. до 2000 г. и с 2000 г. до 2020 г. в Северной Америке, Европе, Японии от объема продаж по всему миру.

In [ ]:
df_games['Период'] = np.where(df_games['Year'] < 2000, 'До 2000', '2000-2020')

sales = df_games.groupby('Период')[['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales']].sum().T
sales.columns = ['Продажи до 2000', 'Продажи после 2000']
sales.index = ['Северная Америка', 'Европа', 'Япония', 'Другие регионы']

In [ ]:
# Диаграмма для 1980–2000
fig_pie6 = go.Figure()
fig_pie6.add_pie(
    labels=sales.index,
    values= sales['Продажи до 2000'].values,
    textinfo='label+percent',
    hole=0.3,
    name='1980–2000'
)
fig_pie6.update_layout(
    title='Доли региональных продаж (1980–2000)',
    title_x=0.5,
    width=900,
    height=400,
    legend_title_text='Регион'
)
fig_pie6.show()

# Диаграмма для 2000–2020
fig_pie7 = go.Figure()
fig_pie7.add_pie(
    labels=sales.index,
    values=sales['Продажи после 2000'].values,
    textinfo='label+percent',
    hole=0.3,
)
fig_pie7.update_layout(
    title='Доли региональных продаж (2000–2020)',
    title_x=0.5,
    width=900,
    height=400,
    legend_title_text='Регион'
)
fig_pie7.show()